In [1]:
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.utils import set_seed
from mingpt.model import GPT
from mingpt.bpe import BPETokenizer

c:\Users\dalli\anaconda3\envs\pydl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
select_mingpt = True
model_architecture = 'gpt2'
device = 'cpu'
set_seed(42)

In [3]:
if select_mingpt:
    model = GPT.from_pretrained(model_architecture)
else:
    model = GPT2LMHeadModel.from_pretrained(model_architecture)
    model.config.pad_token_id = model.config.eos_token_id

model.to(device)
model.eval()

number of parameters: 124.44M


c:\Users\dalli\anaconda3\envs\pydl\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dalli\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


AssertionError: 

In [ ]:
def generate(prompt="", sample_count=10, steps=20, d0_sample=True):
    tokenizer = GPT2Tokenizer.from_pretrained(model_architecture)
    if len(prompt) == 0:
        prompt = '<|endoftext|>'
    
    encoded = tokenizer(prompt, return_tensors='pt').to(device)
    input = encoded["input_ids"].expand(sample_count, -1)

    output = model.generate(input, max_new_tokens=steps, do_sample=d0_sample, top_k=40)

    for i in range(sample_count):
        print(tokenizer.decode(output[i].cpu().squeeze(), skip_special_tokens=True))


In [ ]:
generate(prompt="Math is")